In [1]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re 
from urllib.parse import urljoin
from pymongo import MongoClient
import time


In [2]:
#判斷是否登出
def is_logout(driver):
    soup=bs(driver.page_source,'lxml')
    logout=soup.select('.document__hint_section')
    if len(logout)!=0:
        driver=login(driver)
        time.sleep(3)
        return True,driver
    else:
        return False,driver
    


In [10]:
#解析網頁
def parse(driver,indictment,page_num,db_indict):
    fail_open=[]
    judg_fail_open=[]
    parse_base_url='https://lawsnote.com/'
    db_insert_counter=0
    
    for i in range(0,len(indictment)):
        #合成起訴書完整url
        parse_url=urljoin(parse_base_url,indictment[i])
        driver.get(parse_url)
        try:
            WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME,'main-view')))
            time.sleep(1)
            #判斷有無登出
            is_out,driver=is_logout(driver)
            if not is_out:
                #parse起訴書
                judgement=parse_indictment(driver)
                '''
                #進入判決書
                falg,driver=enter_judgement_page(driver)
                #開啟成功->parse判決書
                if falg==True:
                    j_judgement=parse_judgement(driver)
                #開啟失敗->紀錄在judg_fail->跳下一筆
                elif falg==False:
                    judg_fail_open=judgement['判決書']
                    continue
                '''
                #起訴書、判決書--存入DB
                #judgement.update({'判決法條':j_judgement['相關法條'],'判決法條內文':j_judgement['法條內文']} ) 
                #insert_to_db(judgement,j_judgement)
                insert_to_db(judgement,db_indict)
                #print("insert db")
                db_insert_counter+=1
            else:
                print('登入失敗')
                break
                
        except Exception as e:
            #起訴書開啟失敗-->記錄在fail_open list
            fail_open.append(indictment[i])
            print('起訴書開啟失敗 msg:',e)
            continue  
    print("insert db:",db_insert_counter)
    
    #判斷是否有錯誤紀錄
    if len(fail_open)!=0:
        log_record(False,fail_open,i)
        

In [4]:
def parse_indictment(driver):
    soup=bs(driver.page_source,'lxml')
    #起訴書處理title
    title_label=soup.select('.document__section.document__meta-content')
    title_text=title_label[0].find_all('li')
    judgement={}
    judgement['字號']=title_text[0].text.replace('【裁判字號】','')
    judgement['日期']=title_text[1].text.replace('【裁判日期】','')
    judgement['案由']=title_text[2].text.replace('【裁判案由】','')
    #起訴書處理內文 content_label
    content_label=soup.select('.document__section.document__judgement-content')
    judgement['判決書內文']=content_label[0].text
    #起訴書相關法條
    related_Issue=soup.select('.document-misc__sub-container.document-misc__relatedIssue-container')
    r=related_Issue[0].select('.document-misc__item-container.judgement-misc-content')
    #起訴書相關法條標題 related_Issue_title
    related_Issue_title=[x.find('div').text for x in r]
    #起訴書相關法條內文  {法條名:內文} related_Issue_content
    related_Issue_content={}
    for i in range(len(related_Issue_title)):
        related_Issue_content[related_Issue_title[i]]=r[i].find_all('ul')[0].text

    #起訴書dict key:字號,日期,案由,起訴書內文,相關法條標題:list,相關法條內文:{法條名:內文},判決書
    judgement['相關法條']=related_Issue_title
    judgement['法條內文']=related_Issue_content
    #judg=soup.select('.document-misc__sub-container.document-misc__judgementHistory-container')
    #judgement['判決書']=judg[0].find('a')['id']
    return judgement

In [5]:
#錯誤紀錄
def log_record(is_end,fail_open=None,page_num=None):
    localtime = time.asctime( time.localtime(time.time()) )
    #起訴書開啟失敗紀錄
    with open('fail_open.txt','a') as f:
        if not is_end:
            for i in fail_open:
                f.write("{} 第{}頁  href:{}\n".format(localtime,page_num,i))
        else:
            f.write('-------------------------------------------\n')        
    return

def log_error(page_num,error_url,exp_msg):
    localtime = time.asctime( time.localtime(time.time()) )
    with open('crawler_error.txt','a') as f:
        f.write("{} 第{}頁 href:{} 錯誤msg:{}\n",format(localtime,page_num,error_url,exp_msg))

In [6]:
#DB
def db_connect(mongo_url,port,indic_name):
    try:
        client=MongoClient(mongo_url,port)
        db=client['law_judgement']
        indicit=db[indic_name]
        
    except Exception as e:
        print('db_connect',e)
    return indicit
                    
def insert_to_db(judgement,db_indict):
    try:
        db_indict.insert_one(judgement)
        
    except Exception as e:
        print('db insert fail:',e)
        

In [7]:
def enter_judgement_page(driver):
    soup=bs(driver.page_source,'lxml')
    judg=soup.select('.document-misc__sub-container.document-misc__judgementHistory-container')
    judg[0].find('a')['id']
    driver.find_element_by_link_text(judg[0].find('a')['id']).click()
    try:
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME,'main-view')))
    except:
        return False,driver
    return True,driver


In [8]:
def catch_href(soup):
    sub_item=soup.find_all('a',{'class':'search-result__item-link'})
    return sub_item

def search_fit_indictment(sub_item):
    indictment_item=[]
    for s in sub_item:
        if s.text.find('傷害')!=-1 and s.text.find('判決')!=-1 and s.text.find('刑事')!=-1:
            indictment_item.append(s['href'])
    return indictment_item

def login(driver):
    driver.find_element_by_xpath(u"(.//*[normalize-space(text()) and normalize-space(.)='揪團'])[1]/preceding::button[1]").click()
    #driver.find_element_by_xpath(u"//*[@id=\"root\"]/div/div/header/div[4]/div/button[1]").click()
    driver.find_element_by_id("email").click()
    driver.find_element_by_id("email").clear()
    driver.find_element_by_id("email").send_keys("yushubo9012@gmail.com")
    time.sleep(1)
    driver.find_element_by_id("password").clear()
    driver.find_element_by_id("password").send_keys("asd753")
    time.sleep(1)
    driver.find_element_by_xpath(u"(.//*[normalize-space(text()) and normalize-space(.)='忘記密碼？'])[1]/following::button[1]").click()
    return driver

In [17]:
#main funcation

#變數初始化
#------------------------------------------------------------------------
#要抓取的頁數
page_num=2000
#起始頁
start_page_num=1000
base_url='https://lawsnote.com/search/judgement/%E5%9C%B0%E6%96%B9%E6%B3%95%E9%99%A2%20%E5%88%91%E4%BA%8B%20%E5%82%B7%E5%AE%B3/'
log=False
#------------------------------------------------------------------------
#建立DB連線
db_indict=db_connect('jacky0603.synology.me',28017,'judgememt_4')

driver=webdriver.Chrome(r'C:\Users\user\Desktop\chromedriver.exe')

for i in range(start_page_num,page_num):
    url=urljoin(base_url,str(i))
    driver.get(url)
    try:
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, "search-result__main")))
        time.sleep(3)
        print('第',i,'頁  :')
        #辦斷是否登入
        if log==False:
            driver=login(driver)
            time.sleep(1)
            log=True
        #開始解析
        html=driver.page_source
        soup=bs(html,'lxml')
        #抓出頁面中的判決書連結
        sub_item=catch_href(soup) 
        #篩出府和條件的連結
        indictment=search_fit_indictment(sub_item) 
        #若該頁有符合條件的起訴書-->開始爬起訴書
        if len(indictment)!=0:
            print('indict:',len(indictment),'筆\n')
            parse(driver,indictment,i,db_indict)
            
    except Exception as e:
        log_error(i,url,e)
        driver.close()
    
log_record(True)#結束符號
      

第 1000 頁  :
indict: 10 筆

insert db: 10
第 1001 頁  :
第 1002 頁  :
第 1003 頁  :
第 1004 頁  :
第 1005 頁  :
第 1006 頁  :
第 1007 頁  :
第 1008 頁  :
第 1009 頁  :
第 1010 頁  :
第 1011 頁  :
第 1012 頁  :
第 1013 頁  :
第 1014 頁  :
第 1015 頁  :
第 1016 頁  :
第 1017 頁  :
第 1018 頁  :
第 1019 頁  :
第 1020 頁  :
第 1021 頁  :
第 1022 頁  :
第 1023 頁  :
第 1024 頁  :
第 1025 頁  :
第 1026 頁  :
第 1027 頁  :
第 1028 頁  :
第 1029 頁  :


TypeError: format() takes at most 2 arguments (4 given)